In [413]:
import random
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pickle
import os
import torchaudio

from torchaudio.transforms import MelSpectrogram

N_SAMPLES=25500 #fixed length





class AudioDataset(Dataset):
    
    def __init__(self, data_dir,split='',transform=None):
        self.paths=[]
        self.labels=[]
        self.label_list=['808s','Claps','Closed Hats','Crashes & Cymbals','Extras','FX','Kicks','Open Hats','Percs','Snares','Vox']
        
        for root, subdirs, files in os.walk(data_dir):
            if len(files)>0:
                label=os.path.basename(os.path.normpath(root))
                if label in self.label_list:
                    for file in files:
                        if file[-3:]=='wav' or file[-3:]=='WAV':
                            self.labels.append(label)
                            self.paths.append(os.path.join(root, file))
                            
                            
                            
                            
        c = list(zip(self.paths, self.labels))
        random.Random(42).shuffle(c)
        self.paths, self.labels = zip(*c)
        if split=='val':
            self.paths=self.paths[:len(self.paths)//10]
            self.labels=self.labels[:len(self.paths)//10]
        elif split=='train':
            self.paths=self.paths[len(self.paths)//10:]
            self.labels=self.labels[len(self.paths)//10:]
        
        self.n_samples=len(self.paths)
    
    def __getitem__(self, index):
        waveform, sample_rate = torchaudio.load(self.paths[index], normalize=True)
        waveform = torch.mean(waveform, dim=0).unsqueeze(0) #mono
        #
        # to fixed length
        target = torch.zeros(1, N_SAMPLES)
        target[:,:min(waveform.shape[1],N_SAMPLES) ] = waveform[:,:min(waveform.shape[1],N_SAMPLES)]
        waveform=target
        
        #mel, expand for resnet to 3 channels
        mel = MelSpectrogram(sample_rate)(waveform)

        return  mel.expand(3,-1,-1), torch.tensor(self.label_list.index(self.labels[index]))

   
    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples
    def classes(self):
        return self.label_list

In [423]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torchaudio
cudnn.benchmark = True
plt.ion()   # interactive mode


data_dir =data_dir = './data'
audio_datasets = {x: AudioDataset(data_dir,x)
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(audio_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val']}
dataset_sizes = {x: len(audio_datasets[x]) for x in ['train', 'val']}
class_names = audio_datasets['train'].classes()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [425]:
from tqdm import tqdm
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase],position=0,leave=True):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [426]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [428]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, len(class_names)) #classes

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------


 46%|████████████████████████████████████▉                                           | 117/253 [00:18<00:24,  5.63it/s]